<div>
<img src="https://www.nebrija.com/images/logos/logotipo-universidad-nebrija.jpg" width="200">
</div>

**ALGORITMOS** -
Prof: Carmen Pellicer Lostao

# Algoritmo de Simon

In [ ]:
# initialisation
import numpy as np
import math, cmath

# Importing standard Qiskit libraries
from qiskit.providers.aer import Aer, AerSimulator
from qiskit import QuantumCircuit, transpile, assemble, Aer, IBMQ, execute
from qiskit.providers.ibmq import least_busy, IBMQBackend
from qiskit.tools.jupyter import *

# basic plot tools
from qiskit.visualization import plot_histogram
from qiskit.visualization import plot_bloch_multivector
from qiskit.visualization import array_to_latex

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()


# ORACULOS

En el algoritmo de Simon tenemos una funcion de Oráculo desconocida $f$, que se garantiza que será uno a uno ($1:1$) o dos a uno ($2:1$), donde __uno-a-uno__ y __dos-a-uno__, significa que la funcion tiene las siguientes propiedades:

- **uno-a-uno**: asigna exactamente una salida única para cada entrada. Un ejemplo con una función que toma 4 entradas es:

$$f(1) \rightarrow 1, \quad f(2) \rightarrow 2, \quad f(3) \rightarrow 3, \quad f(4) \rightarrow 4$$

- **dos-a-uno**: asigna exactamente dos entradas a cada salida única. Un ejemplo con una función que toma 4 entradas es:

$$f(1) \rightarrow 1, \quad f(2) \rightarrow 2, \quad f(3) \rightarrow 1, \quad f(4) \rightarrow 2$$

Y donde el mapeo dos-a-uno se realiza según una cadena de bits oculta, $s$, donde:

$$
\textrm{dado }x_1,x_2: \quad f(x_1) = f(x_2) \\
\textrm{está garantizado }: \quad x_1 \oplus x_2 = s
$$

Y resulta que ambos casos se reducen al mismo problema de encontrar $s$, donde una cadena de bits de $s={000...}$ representa el $f$ uno a uno.

Podemos crear una __funcion de oráculo__ para n qubits que codifique una funcion binaria de este tipo.

#### EJERCICIO

Escribe una funcion que tome dos cadenas de bits $x1$ y $x2$ devuelva la cadena $s$ que cumple $x_1 \oplus x_2 = s$


In [ ]:
def xor_bitstrings(bitstring1, bitstring2):
    ''' input are two strings of length n bits
        output one string of n bits with bitstring1 XOR bitstring2 operation
    '''


In [ ]:
# Example usage:
bitstring1 = "00"
bitstring2 = "01"
result = xor_bitstrings(bitstring1, bitstring2)

print(f"The XOR of {bitstring1} and {bitstring2} is: {result}")

## Oráculo para una cadena binaria dada

### EJERCICIO

Dada la siguiente funcion binaria (2:1) de n= 2 bits:

```
x_1 x_0   f(x_1 x_0)$
0   0       0   0
0   1       1   1
1   0       1   1
1   1       0   0
```

Calcula mediante un programa la cadena $b$ que cumple que:

$$
\textrm{dado }x_1,x_2: \quad f(x_1) = f(x_2) \\
\textrm{está garantizado }: \quad x_1 \oplus x_2 = s
$$

In [ ]:
# utiliza la funcion xor anterior y computa la cadena b a partir de los valores de x1 y x2 que cumplen f(x1=f(x2)


Utiliza el Composer para construir un circuito sea un Oráculo de Simon con esta cadena $s$

Ponemos los n qubits del input en superposicion y la ejecutamos para ver que es la funcion que realiza la operacion que hemos definido. Estamos ejecutando todos los valores de input posibles a la vez. 

#### EJERCICIO

Dada una cadena de n=2 bits, construye un circuito con n qubits de entrada y n qubits de salida, que compute el resultado de la funcion (2:1) del oráculo de Simon siguiente:

```
x_1 x_0   f(x_1 x_0)$
0   0       0   0
0   1       0   1
1   0       0   0
1   1       0   1
```

Calcula mediante un programa la cadena $s$ que cumple que:

$$
\textrm{dado }x_1,x_2: \quad f(x_1) = f(x_2) \\
\textrm{está garantizado }: \quad x_1 \oplus x_2 = s
$$

In [1]:
# utiliza la funcion xor anterior y computa la cadena b a partir de los valores de x1 y x2 que cumplen f(x1=f(x2)


Utiliza el Composer para construir un circuito sea un Oráculo de Simon con esta cadena $s$

Ponemos los n qubits del input en superposicion y la ejecutamos para ver que es la funcion que realiza la operacion que hemos definado. Estamos ejecutando todas los valores de input posibles a la vez. 

## Creacion de un oraculo aleatorio

Creamos una funcion de oráculo para un numero de bits n que genera una cadena binaria aleatoria y devuelve un circuito que computa una funcion (2:1) que es un Oraculo de Simon caracterizado por esta cadena

In [ ]:
### HACEMOS LA FUNCOIN DEL ORACULO

def simon_oracle(n):
    #La funcion crea un objeto de QuantumCircuit y lo devuelve
    #El circuito del oraculo tiene n qubit de entrada y uno de salida -> en total n+1 qubits

    oracle_qc = QuantumCircuit(2*n,n)
    #generation of string s and printing it for further comprobations
    s= ''.join([str(np.random.randint(2)) for b in range(n)])
    print('s',s[::-1]) # la cadena caracteristica es esta, puesto que se codifica en el circuito tiene orden invertido de qubits
    
    #draw a barrier before oracle
    oracle_qc.barrier()

    # copy the content of the first register to the second register
    for i in range(n):
        oracle_qc.cx(i, n+i)

    # get the least index j such that s_j is "1"
    j = -1
    for i, c in enumerate(s):
        if c == "1":
            j = i
            break

    # Creating 1-to-1 or 2-to-1 mapping with the j-th qubit of x as control to XOR the second register with s
    for i, c in enumerate(s):
        if c == "1" and j >= 0:
            oracle_qc.cx(j, n+i) #the i-th qubit is flipped if s_i is 1

    # get random permutation of n qubits
    perm = list(np.random.permutation(n))

    #initial position
    init = list(range(n))
    i = 0
    while i < n:
        if init[i] != perm[i]:
            k = perm.index(init[i])
            oracle_qc.swap(n+i, n+k) #swap qubits
            init[i], init[k] = init[k], init[i] #marked swapped qubits
        else:
            i += 1

    # randomly flip the qubit
    for i in range(n):
        if np.random.random() > 0.5:
            oracle_qc.x(n+i)

    # Apply the barrier to mark the end of the blackbox function
    oracle_qc.barrier()


    oracle_qc.name = 'Simon-Oracle' #to show in the display circuit
    return oracle_qc

#### EJERCICIO

Lo ejecutamos en superposicion para ver la salida de la funcion del oráculo

Comprobamos que las cadenas que cumplen $ x_1 \oplus x_2 = s$ son las que tienen el mismo resultado de la funcion del Oráculo

# Ejecucion del Algoritmo de Simon

#### EJERCICIO

Construye un circuito que ejecute el algoritmo de Simon para un Oraculo aleatorio.

Ejecutalo y obten los resultados de las medidas para determinar la cadena s del Oraculo en un proceso posterior de postprocesado.

In [ ]:
#Creamos un circuito de n qubits+1
n=

#ponemos los qubits de entrada en superposicion


# Le añadimos el oráculo con funcion aleatoria


#le añadimos las puertas de Hadamard finales


#ponemos las puertas de medida

    
    
#Mostramos el circuito


#Ejecutamos el circuito


#mostramos histograma resultado


#### EJERCICIO

Realizaremos la etapa de postporcesado del algoritmo para descubrir la cadena s del Oráculo y lo haremos inicialmente con una aproximacion de fuerza bruta.

Construye las siguientes funciones binarias:

- __dot_product_modulo_2__: que tome dos bitstrings y calcule el producto escalar binario (modulo 2) de ambas

- __generate_all_bitstrings__: que tome un numero de bits y genere todos los bitstreams posibles para ese numero de bits dado


Y utilizalas para recorrer todas las posibles cadenas s y encontrar aquellas que cumplen $ s.x mod 2 = 0 $ para todas las cadenas $x$ medidas en la salida del circuito del algoritmo de Simon anterior


In [ ]:
def dot_product_modulo_2(bitstring1, bitstring2):




In [ ]:
# Example usage:
bitstring1 = "1101"
bitstring2 = "1010"
result = dot_product_modulo_2(bitstring1, bitstring2)

print(f"The dot product modulo 2 of {bitstring1} and {bitstring2} is: {result}")

In [ ]:
def generate_all_bitstrings(length):



In [ ]:
# Example usage:
# Set the length of the bitstrings
bitstring_length = 3

# Generate all possible bitstrings of 3 bits
all_bitstrings = generate_all_bitstrings(bitstring_length)

# Print the result
print("All possible bitstrings of length", bitstring_length, ":")
for bitstring in all_bitstrings:
    print(bitstring)

In [ ]:
#calcula todos los valores bitstrings obtenidos con el circuito del algoritmo de Simon son
valores_resultado_simon=

In [ ]:
#Tenemos la cadena del Oráculo, que copiamos aqui para cerciorarnos de que encontramos la misma cadena
s='010101'
print('s',s)

#algoritmo de fuerza bruta que encuentra la cadena s



A continuación se recoge un metodo algebraico para encontrar la cadena s que podemos utilizar tambien en lugar del metodo de fuerza bruta anterior

In [ ]:
# Post-processing step
# Constructing the system of linear equations Y s = 0
# By k[::-1], we reverse the order of the bitstring
lAnswer = [ (k[::-1],v) for k,v in counts.items() if k != "0"*n  ] #excluding the trivial all-zero
#Sort the basis by their probabilities
lAnswer.sort(key = lambda x: x[1], reverse=True)

Y = []
for k, v in lAnswer:
    Y.append( [ int(c) for c in k ] )


#import tools from sympy
from sympy import Matrix, pprint, MatrixSymbol, expand, mod_inverse


Y = Matrix(Y)

#pprint(Y)

#Perform Gaussian elimination on Y
Y_transformed = Y.rref(iszerofunc=lambda x: x % 2==0) # linear algebra on GF(2) 

#to convert rational and negatives in rref of linear algebra on GF(2)
def mod(x,modulus):
    numer, denom = x.as_numer_denom()
    return numer*mod_inverse(denom,modulus) % modulus

Y_new = Y_transformed[0].applyfunc(lambda x: mod(x,2)) #must takecare of negatives and fractional values
#pprint(Y_new)

print("The hidden bistring s[ 0 ], s[ 1 ]....s[",n-1,"] is the one satisfying the following system of linear equations:")
rows, cols = Y_new.shape
for r in range(rows):
    Yr = [ "s[ "+str(i)+" ]" for i, v in enumerate(list(Y_new[r,:])) if v == 1 ]
    if len(Yr) > 0:
        tStr = " + ".join(Yr)
        print(tStr, "= 0")

# Ejecucion en un Simulador con ruido

### EJERCICIO

Ejecuta en un simulador con ruido el circuito con el algoritmo de Simon para ello:

- Creamos el circuito cuántico, que llamamos <b>simon_circuit </b>
- Preparamos un simulador con el ruido de un dispositivo real, el `ibmq_vigo`.
- Transpilamos el circuito y lo ejecutamos en el simulador
- Realizamos el postprocesado de los resultados obtenidos para encontrar la cadena s

Debido al ruido de los calculos no es sencillo ontener un calculo concluyente de la cadena s

In [ ]:
#Creamos un circuito de n qubits+1
n=3
circuit =

#PASO 1 - ponemos los qubits de entrada en superposicion


# PASO 2- Le añadimos el oráculo con funcion aleatoria


#PASO 3 - Le añadimos las puertas Hadamard



#PASO 4- ponemos las puertas de medida solo para los qubits de la entrada



#Mostramos el circuito

Preparamos un simulador con el ruido de un dispositivo real, el `ibmq_vigo`.

In [ ]:
from qiskit.providers.fake_provider import FakeVigo


# Tomamos un backend ruidoso ficticio de fake providers


# creamos el simulador


# Transpilamos el circuito para mapearlo a las puertas ruidosass


# Ejecutamos el circuito y obtenemos el resultado

Y cuando termine la ejecucion, realizamos el postprocesado y obtencion de la cadena $s$ resultado

# Ejecucion en un BackEnd cuántico real

### EJERCICIO

Ejecuta en un backend real cuantico el circuito con el algoritmo de Simon, para ello:

- Creamos el circuito cuántico, que llamamos <b>simon_circuit </b>
- Lo ejecutamos primero en el simulador para ver si funciona bien antes de enviarlo a un BackEnd cuántico real
- Buscamos el backend que este menos cargado de trabajos
- Transpilamos el circuito y lo enviamos para ejecucion
- Realizamos el postprocesado de los resultados obtenidos para encontrar la cadena s

Debido a la imperfeccion de los ordenadores reales y el ruido de los calculos no es sencillo ontener un calculo concluyente de la cadena s

In [ ]:
#Construimos el circuito

#ejecutamos en simulador para comprobar que esta ok

In [ ]:
# buscamos el backend menos ocupado - using a real qc backend: lest busy

In [ ]:
#lanzamos la ejecucion del job

Y cuando termine la ejecucion, realizamos el postprocesado y obtencion de la cadena $s$ resultado